# Transfer learning & fine-tuning
by uramoon@kw.ac.kr<br><br>

Xception 모델을 사용하여 고양이와 개를 분류해봅시다.<br>
Xception의 성능은 아래에서 확인할 수 있습니다.<br>
https://keras.io/api/applications/ <br>
런타임 유형은 GPU로 설정하세요.<br>

처음에는 Xception을 바닥부터 훈련시키고,
나중에는 ImageNet에 대해 훈련된 <br>Xception을 불러와 전이학습을 진행해 두 방법의 결과를 비교해 볼 것입니다.

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/04/15<br>
**Last modified:** 2020/05/12<br>
**Description:** Complete guide to transfer learning & fine-tuning in Keras.<br>
(<a href="https://raw.githubusercontent.com/ronreiter/interactive-tutorials/master/LICENSE">Apache 2.0 License</a>)

## Setup

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 소개

**전이 학습**은 어떤 문제에 대해 학습된 특징들을 새로운 문제의 풀이에 사용하는 것을 뜻합니다. <br>예를 들면, 라쿤을 인식하는 모델에서 사용되는 특징들로 너구리를 인식하는 데 사용할 수 있을 것입니다.<br>

<img src="https://extension.umd.edu/sites/extension.umd.edu/files/styles/optimized/public/2021-02/hgic_veg_wildlife_raccoon.jpg?itok=p4k_Z_CF" height="200"><figcaption>라쿤 사진 출처: https://extension.umd.edu/resource/raccoons</figcaption>

<img src="https://image-notepet.akamaized.net/resize/620x-/seimage/20171108%2Fe6d1ec360a4ab04e21e580882d9c989e.jpg" height="200"><figcaption>너구리 사진 출처: https://www.notepet.co.kr/news/article/article_view/?idx=10434</figcaption>

전이학습은 다음의 수행절차를 갖습니다.
1. 기존에 훈련된 모델을 가져온다.
2. 가져온 모델을 훈련이 불가능하도록 설정한다. (가중치들을 고정시킴)
3. 가져온 모델에 몇 개의 층을 추가한다.
4. 내가 추가한 층만 나의 데이터로 훈련시킨다.

추가적으로 고정시킨 모델도 훈련 가능하도록 설정해 세부 튜닝을 시도할 수 있습니다.

This is adapted from
[Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)
 and the 2016 blog post
["building powerful image classification models using very little
 data"](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

## 데이터 가져오기
원래 25,000장의 그림이 있는데 15,000장만 가져와서 그 중 10,000장은 훈련, 2500장은 검증, 2500장은 테스트에 사용하겠습니다.<br>
(오염된 이미지들이 있어서 다 받아오진 못하고 수 백장의 사진이 걸러집니다.)

In [29]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

# 임시 코드 (다운로드가 안되면 지우세요.)
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Number of training samples: 9305
Number of validation samples: 2326
Number of test samples: 2326


## TODO1: 데이터셋 전처리하기

Xception 모델은 지난 시간 Cats and Dogs 노트북에서 사용한 코드를 그대로 사용합니다.

In [ ]:
# 각 데이터셋의 크기 변경하기
size = (150, 150)

train_ds = train_ds.map(lambda X, y: (tf.image.resize(X, size), y))
validation_ds = validation_ds.map(lambda X, y: (tf.image.resize(X, size), y))
test_ds = test_ds.map(lambda X, y: (tf.image.resize(X, size), y))

In [31]:
# Xception의 전처리 기법 적용하기
# 정규화를 따로 해줄 필요가 없습니다.
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

train_ds = train_ds.map(lambda X, y: (preprocess_input(X), y))
validation_ds = validation_ds.map(lambda X,y:(X/255.0,y))#TODO
test_ds = test_ds.map(lambda X,y:(X/255.0,y))#TODO

In [32]:
# 데이터셋을 32장씩 묶습니다.
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

## TODO2: 데이터 증강 층 만들기<br>


In [33]:
from tensorflow import keras
from tensorflow.keras import layers

#TODO: Cats and Dogs 참조
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

## TODO3: 모델 정의하기

가져온 모델 앞에 입력층과 데이터 층강층을 추가하고 뒤에 예측하는 부분 (MLP와 같은 Dense층) 을 추가합니다.

In [34]:
from keras import models
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.optimizers import Adam
base_model = keras.applications.Xception(
    weights=None, # 가중치가 랜덤하게 설정되어 훈련이 필요한 Xception 모델을 가져옵니다.
    input_shape=(150, 150, 3),
    include_top=False,  # 개와 고양이를 예측할 것이기 때문에 ImageNet 데이터를 예측하는 출력층은 포함하지 않습니다.
)
base_model.trainable = False

model = models.Sequential()

#TODO: model에 입력층 추가
model.add(layers.Input(shape=(150,150,3)))
#TODO: model에 데이터 증강층 추가
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)
model.add(data_augmentation)
#가져온 Xception 모델 추가
model.add(base_model)

# MLP는 일차원으로 펼쳐진 입력이 필요합니다.
# 아래 둘 중 원하는 것 하나만 쓰세요.
# Xception은 5 x 5 사이즈의 2048개 채널을 출력합니다.
#model.add(layers.Flatten()) # 각 채널의 모든 픽셀들을 일차원으로 이어붙이는 무식하고 낡은 방법 (25개 x 2048장)
model.add(layers.GlobalAveragePooling2D()) # 각 5 x 5 채널을 하나의 값으로 요약한 후 일차원으로 이어붙이는 방법 (2048개)

# 출력층 추가 (0~1 출력하는 노드 하나만 필요하고 적절한 활성화 함수 기재)
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))
# 컴파일 (이진 분류 문제에 적합한 손실함수 선택)
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

In [35]:
# 훈련 가능한 파라미터의 수를 확인해 보세요.
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_11 (Sequential)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,123,881 (80.58 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

## 훈련하기
굉장히 시간이 오래 걸리므로 epochs을 3으로 설정하여 훈련해봅시다.

In [36]:
from keras.callbacks import EarlyStopping

epochs = 3
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Epoch 1/3
291/291 ━━━━━━━━━━━━━━━━━━━━ 41s 122ms/step - accuracy: 0.5021 - loss: 0.6932 - val_accuracy: 0.4948 - val_loss: 0.6933
Epoch 2/3
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 95ms/step - accuracy: 0.4896 - loss: 0.6933 - val_accuracy: 0.4948 - val_loss: 0.6933
Epoch 3/3
291/291 ━━━━━━━━━━━━━━━━━━━━ 33s 113ms/step - accuracy: 0.4903 - loss: 0.6933 - val_accuracy: 0.4948 - val_loss: 0.6933


## 테스트 데이터로 평가해보기



In [37]:
model.evaluate(test_ds)

73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.4955 - loss: 0.6933


[0.693320631980896, 0.49398109316825867]

좋은 장비와 충분한 시간이 있다면 성능을 더 끌어올릴 수도 있겠지만 우리에겐 그러한 여력이 없습니다.

## TODO 4: 전이학습 시도하기
Hint: https://keras.io/api/applications/xception/

In [38]:
from keras import models
from keras import layers

base_model = keras.applications.Xception(
    weights="imagenet",#TODO, # ImageNet에 대해 훈련된 신경망을 가져옵니다. 나머지 채우는 부분은 TODO3과 동일
    input_shape=(150, 150, 3),
    include_top=False,  # 개와 고양이를 예측할 것이기 때문에 ImageNet 데이터를 예측하는 출력층은 포함하지 않습니다.
)
# Xception은 훈련이 불가능하도록 설정합니다.
base_model.trainable = False # 나중에 세부 튜닝을 위해서 훈련되도록 설정할 수도 있습니다.

model = models.Sequential()

#TODO: 입력층 추가
model.add(layers.Input(shape=(150,150,3)))

#TODO: 데이터 증강층 추가
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)
model.add(data_augmentation)

#가져온 Xception 모델 추가
model.add(base_model)

# MLP는 일차원으로 펼쳐진 입력이 필요합니다.
# 아래 둘 중 원하는 것 하나만 쓰세요.
# Xception은 5 x 5 사이즈의 2048개 채널을 출력합니다.
#model.add(layers.Flatten()) # 각 채널의 모든 픽셀들을 일차원으로 이어붙이는 무식하고 낡은 방법 (25개 x 2048장)
model.add(layers.GlobalAveragePooling2D()) # 각 5 x 5 채널을 하나의 값으로 요약한 후 일차원으로 이어붙이는 방법 (2048개)

# 출력층 추가 (0~1 출력하는 노드 하나만 필요하고 적절한 활성화 함수 기재)
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))
# 컴파일 (이진 분류 문제에 적합한 손실함수 선택)
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

In [39]:
# 훈련 가능한 파라미터의 수를 확인해 보세요.
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_13 (Sequential)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,123,881 (80.58 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

## TODO 5: 훈련하기
지난 번보다는 빠르지만 어느 정도 시간이 소요되므로 patience를 너무 높게 설정하지는 마세요.<br>
시간이 오래 걸리면 중간에 중단해도 됩니다.

In [40]:
from keras.callbacks import EarlyStopping

# TODO: 자유롭게 설정하세요.
epochs = 20
es = EarlyStopping(patience=2, restore_best_weights=True)
model.fit(train_ds, epochs=epochs, callbacks=es, validation_data=validation_ds)

Epoch 1/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 34s 102ms/step - accuracy: 0.9035 - loss: 0.2277 - val_accuracy: 0.9678 - val_loss: 0.0867
Epoch 2/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 33s 113ms/step - accuracy: 0.9407 - loss: 0.1379 - val_accuracy: 0.9682 - val_loss: 0.0803
Epoch 3/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9421 - loss: 0.1319 - val_accuracy: 0.9682 - val_loss: 0.0786
Epoch 4/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9476 - loss: 0.1234 - val_accuracy: 0.9690 - val_loss: 0.0800
Epoch 5/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9490 - loss: 0.1242 - val_accuracy: 0.9699 - val_loss: 0.0782
Epoch 6/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9535 - loss: 0.1171 - val_accuracy: 0.9690 - val_loss: 0.0765
Epoch 7/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9566 - loss: 0.1115 - val_accuracy: 0.9690 - val_loss: 0.0751
Epoch 8/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9511 - loss: 0.1098 

## 테스트 데이터로 평가해보기

In [41]:
model.evaluate(test_ds)

73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.9684 - loss: 0.0768


[0.08364738523960114, 0.9656062126159668]

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FW0TLL%2FbtqRx0uGeC2%2FKxbOgpwhzXXvwu1VtcmjNK%2Fimg.jpg' height=200>

## 세부 튜닝하기

가져온 모델인 Xception까지 우리 데이터에 맞게 훈련해봅시다.<br>
너무 많은 시간이 소요될 수 있으니 적당히 수행합니다. (중간에 중단 버튼 눌러도 괜찮음)

In [42]:
# 가져온 모델 훈련 가능하도록 설정
base_model.trainable = True

# 훈련 가능한 파라미터 수를 위와 비교해보세요.
model.summary()

# 컴파일 해주세요.


# 방금 훈련한 모델을 조금 더 훈련시킵니다.
epochs = 10
model.fit(train_ds, epochs=epochs, callbacks=es, validation_data=validation_ds)

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_13 (Sequential)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,648,685 (82.58 MB)

 Trainable params: 21,069,353 (80.37 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 524,804 (2.00 MB)

Epoch 1/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 97ms/step - accuracy: 0.9597 - loss: 0.0966 - val_accuracy: 0.9712 - val_loss: 0.0800
Epoch 2/10
291/291 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - accuracy: 0.9573 - loss: 0.1058 - val_accuracy: 0.9716 - val_loss: 0.0764


## 다시 테스트 데이터로 평가해보기

In [43]:
model.evaluate(test_ds)

73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.9687 - loss: 0.0815


[0.09072946757078171, 0.9677557945251465]

적은 시간을 사용했을 경우 오히려 성능이 떨어질 가능성이 높습니다.<br>
원본 노트북에서는 훈련 불가능하게 만든 상태에서 20 epochs, 세부 튜닝에서 10 epochs 훈련하는데 성능이 조금 좋아집니다.<br>
전이학습에서 세부 튜닝하는 방법까지 공부한 것에 의의를 둡시다.